# Stage ingénieur NLP - Test Technique
Ettore Hidoux, Janvier 2023

# Instructions
La démarche de recherche et l'utilisation du code est présentée dans le notebook `lab.ipynb` et le code répondant aux différentes questions est dans le fichier `lab.py`.

Notre logiciel permet d’effectuer des recherches et de visualiser les articles de presse sur un territoire donné. Les recherches peuvent être effectuées en saisissant manuellement des mots-clefs dans une barre de recherche ou en sélectionnant un thème.
Une des problématiques rencontrée est la présence de doublons dans nos articles.

**Note**: Un article de presse peut se retrouver en plusieurs exemplaires dans les résultats de la recherche et conduire à une perception dégradée de la qualité du logiciel pour le client. Ces doublons peuvent avoir plusieurs origines :

1. plusieurs éditions d’un même journal (Ouest France - Edition Sud Vendée, Ouest France - Edition Loire Atlantique…) publient le même article, qui se retrouve plusieurs fois dans nos données ;
2. un article est publié dans plusieurs journaux partageant un même éditeur (Var Matin, Nice Matin par exemple) ;
3. un article est publié, puis édité et nous est donc envoyé deux fois, dans sa version originale puis dans sa version éditée ;
4. un article est transmis en plusieurs formats avec le même contenu légèrement modifié (titres différents, certains paragraphes en moins…) ;
  
**Objectif**: Nous souhaiterions détecter et éliminer ces doublons.
L’objectif du test consistera à proposer une démarche d’identification des doublons au sein d’une liste d’articles de presse.
Pour simplifier, on considèrera qu’un article est une structure de données (dictionnaire python, objet, …) comportant les champs/attributs suivants :

- id : l’identificant l’exemplaire (les doublons du même article possèdent des identifiants différents)
- date_de_publication : la date de publication de l’article (exemple '2021-10-29')
- titre : le titre de l’article
- texte : le contenu (corps) de l’article

## Import du code depuis `lab.py`

* On importe le fichier `.py` situé dans le même fichier que ce notebook.
* On utilise `autoreload` notebook extension pour que les modifications du fichier `lab.py` soient immédiatement disponible sur le notebook. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from test import *

In [3]:
import os
import io
import pandas as pd
import numpy as np

# Question 0 - Etude du dataset

- Ajout d'une ligne pour avoir un triple doublon afin d'être sur que tout est bien supprimé.
- On mélange le dataset pour avoir tout les configurations (pas toujours supprimer le deuxième pareil).
- Comme on va travailler sur une liste d'ids (c'est l'output souhaité), création d'une fonction pour selectionner des infos du dataset en fonction de l'id et des colonnes.

In [4]:
df = pd.read_json('./data/input.json')
new = df.loc[2, :].copy()
new.id = '7d85234a2c4055bd78ab39f5176a9577'
new.date_de_publication = '2021-09-12'
df = df.append(new).reset_index(inplace=False, drop=True)
df.date_de_publication = pd.to_datetime(df.date_de_publication)
df = df.sample(frac=1).reset_index(drop=True)
df

,id,titre,texte,date_de_publication
0,5a730bc8e1c762d7535314c6e74f7b42,"La BEI, la banque européenne qui se chauffe po...","La BEI, la banque européenne qui se chauffe po...",2021-11-01
1,6d834976cb9f73ef7ad97649d6a6659b,"Entre Caumont et Chérienne, l'implantation d'u...",Le Conseil d'État doit rendre sa décision défi...,2021-09-26
2,35f28a0ba422f5b7d1ccb47a5a1995b5,La littérature vaine aux Bibliothèques Idéales,La littérature vaine aux Bibliothèques Idéales...,2021-09-11
3,94d3075509c18863da530cf66ee9d8d7,Matour Des travaux au centre-bourg,"Depuis plusieurs semaines, les travaux au cent...",2021-09-27
4,d124cab43bed3364cd89bb202eb6b813,Quelques 12 septembre,Quelques 12 septembre\n\n1494 naissance du roi...,2021-09-11
5,3cdab92b6507eb01ca1a039e648de497,"Entre Caumont et Chérienne, l'implantation d'u...",Le Conseil d'État doit rendre sa décision défi...,2021-09-26
6,7d85234a6b7055bd78ab39f5176a9577,La littérature vaine aux Bibliothèques Idéales,La littérature vaine aux Bibliothèques Idéales...,2021-09-11
7,bd4c53f55cca9e4e87cdeb47f44e3d09,Quelques 12 septembre,Quelques 12 septembre\n\n1494 naissance du roi...,2021-09-12
8,acdc416b2c4fa25d6d91d9257408f6dc,Des travaux au centre-bourg,"Depuis plusieurs semaines, les travaux au cent...",2021-09-27
9,580a7eb5b805adfba3385d648cc0ec53,"La BEI, la banque européenne QUI SE CHAUFFE PO...","La BEI, la banque européenne QUI SE CHAUFFE PO...",2021-11-01


In [5]:
# check if text can be totally equal
df.loc[2, 'texte'] == df.loc[3, 'texte']
# check if row can be equal (title, date and text)
df.loc[2, :] == df.loc[3, :]

id                     False
titre                  False
texte                  False
date_de_publication    False
dtype: bool

In [6]:
# create a list with article ids 
text_ids = list(df.id)
text_ids

['5a730bc8e1c762d7535314c6e74f7b42',
 '6d834976cb9f73ef7ad97649d6a6659b',
 '35f28a0ba422f5b7d1ccb47a5a1995b5',
 '94d3075509c18863da530cf66ee9d8d7',
 'd124cab43bed3364cd89bb202eb6b813',
 '3cdab92b6507eb01ca1a039e648de497',
 '7d85234a6b7055bd78ab39f5176a9577',
 'bd4c53f55cca9e4e87cdeb47f44e3d09',
 'acdc416b2c4fa25d6d91d9257408f6dc',
 '580a7eb5b805adfba3385d648cc0ec53',
 '7d85234a2c4055bd78ab39f5176a9577']

In [7]:
# function to select row of the dataset by id 
def selectById(id, col=None):
    if col == None:
        return df.loc[df.id == id, :].reset_index(inplace=False, drop=True)
    else:
        if type(col)==list:
            return df.loc[df.id == id, col].reset_index(inplace=False, drop=True)
        else:
            return df.loc[df.id == id, col].values[0]
    
selectById('580a7eb5b805adfba3385d648cc0ec53', 'texte')
selectById('7d85234a2c4055bd78ab39f5176a9577', ['texte', 'titre'])

,texte,titre
0,La littérature vaine aux Bibliothèques Idéales...,La littérature vaine aux Bibliothèques Idéales


In [8]:
(selectById('5a730bc8e1c762d7535314c6e74f7b42', ['texte', 'titre']) == selectById('7d85234a2c4055bd78ab39f5176a9577', ['texte', 'titre'])).sum(1).values[0]

0

# Question 1 - Doublons exacts
Proposer un script permettant d'écarter les doublons exacts. 

Il peut y avoir plus de deux exemplaires d’un même article, votre script doit en tenir compte, c’est-à-dire qu’en sortie, on ne veut qu’un simple exemplaire de l’article qu’il y ait 1 ou 6 doublons.

**Explication:**
On parcourt la liste d'id d'articles une premiere fois puis une deuxième en excluant ceux déjà parcourut. Si le titre et le text sont exactement les mêmes on retire le second de la liste (simplicité, supprimer le premier demande de sortir de la boucle et ajoute une ligne `break`).

In [9]:
text_ids = list(df.id)
for id1 in text_ids:
    #print(id1)
    for id2 in text_ids[text_ids.index(id1)+1:]:
        #print(' ', id2)
        is_equal = (selectById(id1, ['texte', 'titre']) == selectById(id2, ['texte', 'titre']))
        if is_equal.sum(1).values[0] == 2:
            text_ids.remove(id2)
            print(id1, id2)
            
print(len(text_ids))  

35f28a0ba422f5b7d1ccb47a5a1995b5 7d85234a6b7055bd78ab39f5176a9577
35f28a0ba422f5b7d1ccb47a5a1995b5 7d85234a2c4055bd78ab39f5176a9577
d124cab43bed3364cd89bb202eb6b813 bd4c53f55cca9e4e87cdeb47f44e3d09
8


In [10]:
removeExactDuplicates(df)

['5a730bc8e1c762d7535314c6e74f7b42',
 '6d834976cb9f73ef7ad97649d6a6659b',
 '35f28a0ba422f5b7d1ccb47a5a1995b5',
 '94d3075509c18863da530cf66ee9d8d7',
 'd124cab43bed3364cd89bb202eb6b813',
 '3cdab92b6507eb01ca1a039e648de497',
 'acdc416b2c4fa25d6d91d9257408f6dc',
 '580a7eb5b805adfba3385d648cc0ec53']

# Question 2 - Doublons exacts ou presque
Généraliser votre script pour écarter les doublons exacts ou presque. La règle de priorité est de :

1. garder l’article le plus récent
2. si tous les articles similaires sont publiés le même jour, garder le titre le plus court
3. si tous les articles ont le même titre, le choix est arbitraire

**Explications:**

- Premeierement, j'ai créé une function qui, à partir de deux ids, revoient l'id de l'article à suprimer de la liste en respectant les consignes ci-dessus.
- Ensuite, j'ai utilisé la librairie `difflib` (très utile pour le versionning) et notamment la classe `SequenceMatcher` pour obtenir les séquence de characteres qui correspondent d'un article à l'autre.
- Le score est obtenue par: (2*M)/T, ou M est la somme des taille des séquences matchées et T la somme des tailles des deux articles.
- Pour finir, en utilisant la double boucle précédante, si le score est supérieur à 0.9 (valeur arbitraire définie dans notre cas pour que çà marche, une étude avec plus d'article serait souhaitable pour déterminer ce seuil plus précisement), on utilise la première fonction pour savoir quelle article retirer et le retirer.
- Dans le cas de ce petit dataset, le temps de calcul du score n'est pas important mais à grande échelle il serait possible d'utiliser `real_quick_ratio` qui est 6x plus rapide.
- D'autre librairies existent pour calculer le score de ressemblance, en utilisant d'autres aspect, il est également possible de le faire en etudiant la fréquence d'apparition des mots, ou encore, des n-grams. (le score utilisant la fréquence des mots a été réalisé à la suite mais un peut moins performant en quantité de code mais semble plus rapide, à vérifier sur un plus grand échantillon de données) 

In [27]:
def article2Remove(id1, id2):
    if df.loc[id1, 'date_de_publication'] == df.loc[id2, 'date_de_publication']:
        return int(len(df.loc[id1, 'titre']) >= len(df.loc[id2, 'titre']))
    else:
        return int(
            df.loc[id1, 'date_de_publication'] > df.loc[id2, 'date_de_publication'])
       
article2Remove(0, 1)     

1

In [12]:
import difflib
import io



text1 = selectById('5a730bc8e1c762d7535314c6e74f7b42', ['texte']).values[0]
text2 = selectById('580a7eb5b805adfba3385d648cc0ec53', ['texte']).values[0]
d = difflib.Differ()
diffs = [x for x in d.compare(io.StringIO(text1[0]).readlines(), io.StringIO(text2[0]).readlines()) if x[0] in ('+', '-')]
print(len(diffs))

24


In [13]:
text1[0][:50]

'La BEI, la banque européenne qui se chauffe pour l'

In [14]:
%%time
difflib.SequenceMatcher(None, text1[0], text2[0]).ratio()

CPU times: user 8.26 ms, sys: 219 µs, total: 8.48 ms
Wall time: 8.31 ms


0.9237445753254805

In [15]:
%%time
difflib.SequenceMatcher(None, text1[0], text2[0]).quick_ratio()

CPU times: user 1.61 ms, sys: 3 µs, total: 1.61 ms
Wall time: 1.62 ms


0.9511779293242405

In [16]:
%%time
difflib.SequenceMatcher(None, text1[0], text2[0]).real_quick_ratio()

CPU times: user 629 µs, sys: 2 µs, total: 631 µs
Wall time: 632 µs


0.9514879107253564

```python
import spacy
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")

doc1 = nlp(text1)
doc2 = nlp(text2)
doc3 = nlp(selectById('acdc416b2c4fa25d6d91d9257408f6dc', ['texte']).values[0])

print(doc1.similarity(doc2)) 
print(doc1.similarity(doc3))
```

In [17]:
text_ids, to_remove = list(df.id), []
for id1 in text_ids:
    for id2 in text_ids[text_ids.index(id1)+1:]:
        if (id1 not in to_remove) & (id2 not in to_remove):
            #print('         ', id2)
            text1 = selectById(id1, ['texte']).values[0]
            text2 = selectById(id2, ['texte']).values[0]     
            ratio = difflib.SequenceMatcher(None, text1[0], text2[0]).ratio()
            if ratio >= 0.9:
                print(id1, id2)
                print(ratio)
                #print(selectById(id1, ['titre', 'date_de_publication']))
                #print(selectById(id2, ['titre', 'date_de_publication']))
                print(articleToRemove(df, id1, id2))
                to_remove.append(articleToRemove(df, id1, id2))
                if articleToRemove(df, id1, id2)==id1:
                    print('here')
                    break
for id in to_remove:
    text_ids.remove(id)
            
print(len(text_ids))  

5a730bc8e1c762d7535314c6e74f7b42 580a7eb5b805adfba3385d648cc0ec53
0.9237445753254805
580a7eb5b805adfba3385d648cc0ec53
35f28a0ba422f5b7d1ccb47a5a1995b5 7d85234a6b7055bd78ab39f5176a9577
1.0
7d85234a6b7055bd78ab39f5176a9577
35f28a0ba422f5b7d1ccb47a5a1995b5 7d85234a2c4055bd78ab39f5176a9577
1.0
35f28a0ba422f5b7d1ccb47a5a1995b5
here
94d3075509c18863da530cf66ee9d8d7 acdc416b2c4fa25d6d91d9257408f6dc
1.0
acdc416b2c4fa25d6d91d9257408f6dc
d124cab43bed3364cd89bb202eb6b813 bd4c53f55cca9e4e87cdeb47f44e3d09
1.0
d124cab43bed3364cd89bb202eb6b813
here
6


In [18]:
%%time 
removeDuplicates(df)

CPU times: user 128 ms, sys: 1.25 ms, total: 129 ms
Wall time: 128 ms


['5a730bc8e1c762d7535314c6e74f7b42',
 '6d834976cb9f73ef7ad97649d6a6659b',
 '94d3075509c18863da530cf66ee9d8d7',
 '3cdab92b6507eb01ca1a039e648de497',
 'bd4c53f55cca9e4e87cdeb47f44e3d09',
 '7d85234a2c4055bd78ab39f5176a9577']

In [19]:
# pour obtenir la fréquence des mot
from collections import Counter 
# pour nettoyer le texte et retirer les accents et caractères spéciaux
from unidecode import unidecode
# pour nettoyer le texte et retirer les stopwords
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop


#print(fr_stop)

txt1, txt2 = text1[0], text2[0]
for p in ['\n', '\xa0', '.', ':', ',', '!', '?', ';', '«', '»', '(', ')', '’']:
    txt1 = txt1.replace(p, ' ')
    txt2 = txt2.replace(p, ' ')
    
txt1, txt2 = unidecode(txt1.lower()).split(), unidecode(txt2.lower()).split()
    
for w1, w2 in zip(txt1, txt2):
    if w1 in fr_stop:
        txt1.remove(w1)
    if w2 in fr_stop:
        txt2.remove(w2)
        
freq1, freq2 = Counter(txt1), Counter(txt2)   
words = list(freq1.keys())
words.extend(list(freq2.keys()))
words = list(set(words))

print(len(words))

diff = []
some = []
for word in words:
    # print(freq1[word], freq2[word])
    diff.append(abs(freq1[word]-freq2[word]))
    some.append(freq1[word])
    some.append(freq2[word])
    

print(sum(diff))
print(sum(some))
print(1-(sum(diff)/sum(some)))

343
425
473
0.10147991543340384


In [22]:
wordFrequencyScore(text1[0], text2[0])

0.02863961813842486

In [21]:
%%time
text_ids = list(df.id)
for id1 in text_ids:
    print(id1)
    for id2 in text_ids[text_ids.index(id1)+1:]:
        text1 = selectArticleById(df, id1, ['texte']).values[0]
        text2 = selectArticleById(df, id2, ['texte']).values[0]     
        ratio = wordFrequencyScore(text1[0], text2[0])
        if ratio >= 0.9:
            text_ids.remove(articleToRemove(df, id1, id2))
            if articleToRemove(df, id1, id2) == id1:
                break
            
print(text_ids)

5a730bc8e1c762d7535314c6e74f7b42
6d834976cb9f73ef7ad97649d6a6659b
35f28a0ba422f5b7d1ccb47a5a1995b5
d124cab43bed3364cd89bb202eb6b813
bd4c53f55cca9e4e87cdeb47f44e3d09
acdc416b2c4fa25d6d91d9257408f6dc
7d85234a2c4055bd78ab39f5176a9577
['5a730bc8e1c762d7535314c6e74f7b42', '6d834976cb9f73ef7ad97649d6a6659b', '94d3075509c18863da530cf66ee9d8d7', '3cdab92b6507eb01ca1a039e648de497', 'bd4c53f55cca9e4e87cdeb47f44e3d09', 'acdc416b2c4fa25d6d91d9257408f6dc', '7d85234a2c4055bd78ab39f5176a9577']
CPU times: user 65.4 ms, sys: 1.52 ms, total: 66.9 ms
Wall time: 66 ms


Le principe serait le même avec des n-grams, la différence serait apres le split, au lieu de conserver des mots seuls on les assembleraient par groupe de n.

# Question 3 - Doublons inclusifs
Généraliser votre script pour écarter les doublons inclusifs. La règle de priorité est de :
- garder l’article le plus long

In [24]:
text_ids = list(df.id)
for id1 in text_ids:
    #print(id1)
    for id2 in text_ids[text_ids.index(id1)+1:]:
        #print(' ', id2)
        text1 = selectById(id1, ['texte']).values[0]
        text2 = selectById(id2, ['texte']).values[0]     
        ratio = difflib.SequenceMatcher(None, text1[0], text2[0]).ratio()
        if ratio >= 0.9:
            print(id1, id2)
            print(ratio)
            #print(selectById(id1, ['titre', 'date_de_publication']))
            #print(selectById(id2, ['titre', 'date_de_publication']))
            print(articleToRemove(df, id1, id2))
            text_ids.remove(articleToRemove(df, id1, id2))
        elif ratio >= 0.5:
            print(id1, id2)
            print(ratio)
            text1 = len(selectById(id1, ['texte']).values[0])
            text2 = len(selectById(id2, ['texte']).values[0])
            id = id2
            if text1 >= text2:
                id = id1
            text_ids.remove(id)

            
print(len(text_ids)) 

5a730bc8e1c762d7535314c6e74f7b42 580a7eb5b805adfba3385d648cc0ec53
0.9237445753254805
580a7eb5b805adfba3385d648cc0ec53
6d834976cb9f73ef7ad97649d6a6659b 3cdab92b6507eb01ca1a039e648de497
0.550180980585719
94d3075509c18863da530cf66ee9d8d7 acdc416b2c4fa25d6d91d9257408f6dc
1.0
acdc416b2c4fa25d6d91d9257408f6dc
d124cab43bed3364cd89bb202eb6b813 bd4c53f55cca9e4e87cdeb47f44e3d09
1.0
d124cab43bed3364cd89bb202eb6b813
7d85234a6b7055bd78ab39f5176a9577 7d85234a2c4055bd78ab39f5176a9577
1.0
7d85234a6b7055bd78ab39f5176a9577
6


In [25]:
removeDuplicatesAndInsides(df)

['5a730bc8e1c762d7535314c6e74f7b42',
 '94d3075509c18863da530cf66ee9d8d7',
 '3cdab92b6507eb01ca1a039e648de497',
 'bd4c53f55cca9e4e87cdeb47f44e3d09',
 '7d85234a2c4055bd78ab39f5176a9577']

In [26]:
df.loc[df.id.isin(removeDuplicatesAndInsides(df)), :]

,id,titre,texte,date_de_publication
0,5a730bc8e1c762d7535314c6e74f7b42,"La BEI, la banque européenne qui se chauffe po...","La BEI, la banque européenne qui se chauffe po...",2021-11-01
3,94d3075509c18863da530cf66ee9d8d7,Matour Des travaux au centre-bourg,"Depuis plusieurs semaines, les travaux au cent...",2021-09-27
5,3cdab92b6507eb01ca1a039e648de497,"Entre Caumont et Chérienne, l'implantation d'u...",Le Conseil d'État doit rendre sa décision défi...,2021-09-26
7,bd4c53f55cca9e4e87cdeb47f44e3d09,Quelques 12 septembre,Quelques 12 septembre\n\n1494 naissance du roi...,2021-09-12
10,7d85234a2c4055bd78ab39f5176a9577,La littérature vaine aux Bibliothèques Idéales,La littérature vaine aux Bibliothèques Idéales...,2021-09-12


Ce qui correspond bien à la liste contenue dans `output.json`.

# Question 4 - ML model 
Proposer (sans l’implémenter) une approche utilisant le machine learning pour détecter automatiquement les articles similaires. Vous décrirez une stratégie pour constituer un dataset d’entraînement, les features que vous pensez inclure dans votre modèle et la stratégie d'évaluation du modèle. 

## Part 1 - Rassembler des données 

Il faut constituer un dataset équivalent au precédent, celui utilisé dans cette exercice.

## Part 2 - Transformer ces données

Ensuite plusieurs étapes sont nécessaires (entre autre un algorithme de machine learning nécessite des valeurs numériques):

1. Créer un nouveau dataset:
   - colonne 1: article 1 (seulement le texte ou le titre, retour à la ligne deux fois plus le texte)
   - colonne 2: article 2 (seulement le texte ou le titre, retour à la ligne deux fois plus le texte)
   - label: ça peut être 1 ou 0 pour doublon ou pas mais aussi 0, 1 ou 2 pour pas doublon, doublon ou inclus
<br><br>

2. On ne peut pas garder les deux premières colonnes sous forme de texte, on doit donc les transformer:
   - features simples: taille, nombre de mots, ...
   - features croisées: scores de comparaison des fréquence de mots ou même de n-grams.
   - scores de matching: difflib, fuzzywuzzy, pairwise distances with sklearn, ...
   - cf https://medium.springboard.com/identifying-duplicate-questions-a-machine-learning-case-study-37117723844 pour les idées de métriques à utiliser 

## Part 3 - Entrainer un modèle 

1. Serparer le dataset en deux (coefficient 0.25 à 0.3) pour obtenir un dataset d'entrainement et un de test 
2. Ensuite, on peut utilisé plusieurs modèles de machine learning de classification offert par scikit learn (regression logistique, decision tree, random forest, ...) cf https://scikit-learn.org/stable/supervised_learning.html#supervised-learning ou encore tensorflow ou pytorch pour faire du deep learning et aller plus loin.
3. On entraine le modèle avec le dataset d'entrainement.
4. On teste le modèle avec le dataset d'entrainement. 

## Part 4 - Hypertuning 

1. Après avoir selection, le ou les modèles les plus performants.
2. On peut faire varier les paramètres du modèle pour essayer d'améliorer le score finale (avec un GridSearch par example).
